# N-gram 标注

## 一元标注（Unigram Tagging）

一元标注器（[nltk.UnigramTagger](https://www.nltk.org/_modules/nltk/tag/sequential.html#UnigramTagger)）基于一个简单的统计算法：对每个标识符分配这个独特的标识符最有可能的标记。例如：它将分配标记 JJ 给词 frequent 的所有出现，因为 frequent 用作一个形容词（a frequent word）比用作一个动词（I frequent this cafe）更常见。

[NgramTagger](https://www.nltk.org/_modules/nltk/tag/sequential.html#NgramTagger) 一般接受两种输入：一种是标记好的训练数据，如下例所示；一种是已经经过统计的模型，参照 5.4 节的查询标注器。

In [1]:
import nltk
from nltk.corpus import brown
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')
unigram_tagger = nltk.UnigramTagger(brown_tagged_sents)
print(unigram_tagger.tag(brown_sents[2007]))
print(unigram_tagger.evaluate(brown_tagged_sents))

[('Various', 'JJ'), ('of', 'IN'), ('the', 'AT'), ('apartments', 'NNS'), ('are', 'BER'), ('of', 'IN'), ('the', 'AT'), ('terrace', 'NN'), ('type', 'NN'), (',', ','), ('being', 'BEG'), ('on', 'IN'), ('the', 'AT'), ('ground', 'NN'), ('floor', 'NN'), ('so', 'QL'), ('that', 'CS'), ('entrance', 'NN'), ('is', 'BEZ'), ('direct', 'JJ'), ('.', '.')]
0.9349006503968017


## 分离训练和测试数据

我们必须小心标注器不要在和训练数据相同的测试数据上测试，一个只是记忆训练数据，而不试图建立一个一般模型的标注器会得到一个完美的得分，但在标注新的文本时将是无用的。

相反，我们应该分割数据，90% 为测试数据，其余 10% 为测试数据。

In [2]:
size = int(len(brown_tagged_sents) * 0.9)
train_sents, test_sents = brown_tagged_sents[:size], brown_tagged_sents[size:]
unigram_tagger = nltk.UnigramTagger(train_sents)
print(unigram_tagger.evaluate(test_sents))

0.8121200039868434


## 一般的 N-gram 标注

一个 n-gram 标注器是一个 unigram 标注器的一般化，它的上下文是当前词和它前面 n - 1 个标识符的词性标记。如下图所示，这里我们选择 n = 3，即考虑当前词的前两个词的标记，要选择的是圆圈里的标记，灰色阴影是上下文。

注意： n-gram 标注器不考虑跨越句子边界的上下文，一个句子就是一个词的链表，对于句子开始的词，其前面词的标记都设置为 None。

![tag-context.png](resources/tag-context.png)

现在我们来训练一个 bigram 标注器：

In [3]:
bigram_tagger = nltk.BigramTagger(train_sents)
print(bigram_tagger.tag(brown_sents[2007]))
unseen_sent = brown_sents[4203]
print(bigram_tagger.tag(unseen_sent))
print(bigram_tagger.evaluate(test_sents))

[('Various', 'JJ'), ('of', 'IN'), ('the', 'AT'), ('apartments', 'NNS'), ('are', 'BER'), ('of', 'IN'), ('the', 'AT'), ('terrace', 'NN'), ('type', 'NN'), (',', ','), ('being', 'BEG'), ('on', 'IN'), ('the', 'AT'), ('ground', 'NN'), ('floor', 'NN'), ('so', 'CS'), ('that', 'CS'), ('entrance', 'NN'), ('is', 'BEZ'), ('direct', 'JJ'), ('.', '.')]
[('The', 'AT'), ('population', 'NN'), ('of', 'IN'), ('the', 'AT'), ('Congo', 'NP'), ('is', 'BEZ'), ('13.5', None), ('million', None), (',', None), ('divided', None), ('into', None), ('at', None), ('least', None), ('seven', None), ('major', None), ('``', None), ('culture', None), ('clusters', None), ("''", None), ('and', None), ('innumerable', None), ('tribes', None), ('speaking', None), ('400', None), ('separate', None), ('dialects', None), ('.', None)]
0.10206319146815508


需要注意的是，bigram 标注器对于没见过句子表现很差，只要遇到一个新词，就无法给它分配标记。在上面的例子中，从 million 这个词开始，后面的词都被标注为 None，这是因为在训练过程中从来没有见过前面有 None 标记的这些词。所以它的整体准确率得分非常低。

## 组合标注器

使用更准确的算法在很多时候往往落后于使用更广覆盖范围的算法。例如：我们通过 **backoff** 参数组合 bigram 标注器、unigram 标注器和一个默认标注器：

1. 尝试使用 bigram 标注器标注标识符。
2. 如果 bigram 标注器无法找到一个标记，尝试 unigram 标注器。
3. 如果 unigram 标注器也无法找到一个标记，使用默认标注器。

In [4]:
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)
print(t2.evaluate(test_sents))

0.8452108043456593


我们还可以进一步指定一个标注器需要看到一个上下文的多个实例才能保留它。例如：nltk.BigramTagger(sents, cutoff=2, backoff=t1) 将会丢弃那些只看到一次或两次的上下文。

## 标注生词

我们标注生词的方法仍然是回退到一个正则表达式标注器或一个默认标注器，这些都无法利用上下文。因此，如果我们的标注器遇到词 blog，它会分配相同的标记，不论这个词出现的上下文是 the blog 还是 to blog。

一个有用的基于上下文标注生词的方法是限制一个标注器的词汇表为最频繁的 n 个词，然后使用 5.3 节中的方法替代每个其他的词为一个特殊的词 UNK。训练时，n-gram 标注器会检测 UNK 标记的上下文，例如：如果前面的词是 to（标注为 TO），那么 UNK 可能会被标注为一个动词。

## 存储标注器

在大语料库上训练一个标注器可能需要大量的时间，因此我们可以将训练好的标注器保存到一个文件中以后重复使用。

In [5]:
from pickle import dump
output = open('t2.pkl', 'wb')
dump(t2, output, -1)
output.close()

接着，我们可以在一个单独的 Python 进程中载入保存的标注器：

In [6]:
from pickle import load
input = open('t2.pkl', 'rb')
tagger = load(input)
input.close()

最后，我们检查标注器是否正常工作：

In [7]:
text = """The board's action shows what free enterprise
is up against in our complex maze of regulatory laws ."""
tokens = text.split()
print(tagger.tag(tokens))

[('The', 'AT'), ("board's", 'NN$'), ('action', 'NN'), ('shows', 'NNS'), ('what', 'WDT'), ('free', 'JJ'), ('enterprise', 'NN'), ('is', 'BEZ'), ('up', 'RP'), ('against', 'IN'), ('in', 'IN'), ('our', 'PP$'), ('complex', 'JJ'), ('maze', 'NN'), ('of', 'IN'), ('regulatory', 'NN'), ('laws', 'NNS'), ('.', '.')]


## 性能限制

一个 n-gram 标注器的性能上限是什么？我们可以根据经验来决定这个问题的答案：

In [8]:
cfd = nltk.ConditionalFreqDist(
            ((x[1], y[1], z[0]), z[1])
            for sent in brown_tagged_sents
            for x, y, z in nltk.trigrams(sent))
ambiguous_contexts = [c for c in cfd.conditions() if len(cfd[c]) > 1]
print(sum(cfd[c].N() for c in ambiguous_contexts) / cfd.N())

0.049297702068029296


可以看出，1/20 的 trigrams 是有歧义的。因此给定当前单词及前两个标记，根据训练数据，在 5% 的情况下，有一个以上的标记可能合理地分配给当前词，这是 trigram 标注器的上限。

调查标注器性能的另一种方法是研究它的错误。有些标记可能会比别的更难分配，可能需要专门对这些数据进行预处理或后处理。一个方便的方式查看标注错误是**混淆矩阵**，它用图标表示预期的标记（黄金标准）与实际由标注器产生的标记之间的差异：

In [9]:
test_tags = [tag for sent in brown.sents(categories='editorial')
                for (word, tag) in t2.tag(sent)]
gold_tags = [tag for (word, tag) in brown.tagged_words(categories='editorial')]
print(nltk.ConfusionMatrix(gold_tags, test_tags))

           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            